In [1]:
import random
from sympy import symbols, mod_inverse

# إنشاء كثير حدود عشوائي من الدرجة k-1
# الحد الثابت هو السر
def generate_polynomial(secret, k, prime):
    coeffs = [secret]  # الحد الثابت هو السر
    for _ in range(k - 1):
        coeffs.append(random.randint(1, prime-1))  # باقي الحدود عشوائية
    return coeffs

# تقييم كثير الحدود عند x باستخدام الحسابات المودولو
def evaluate_polynomial(coeffs, x, prime):
    result = 0
    for power, coef in enumerate(coeffs):
        result = (result + coef * pow(x, power, prime)) % prime
    return result

# إنشاء n حصة
def generate_shares(secret, k, n, prime):
    coeffs = generate_polynomial(secret, k, prime)
    shares = []
    for i in range(1, n+1):
        shares.append((i, evaluate_polynomial(coeffs, i, prime)))
    return shares

# إعادة بناء السر باستخدام طريقة لاجرانج
def reconstruct_secret(shares, prime):
    secret = 0
    k = len(shares)
    for j, (xj, yj) in enumerate(shares):
        numerator, denominator = 1, 1
        for m, (xm, _) in enumerate(shares):
            if m != j:
                numerator = (numerator * (-xm)) % prime
                denominator = (denominator * (xj - xm)) % prime
        lagrange_coefficient = (numerator * mod_inverse(denominator, prime)) % prime
        secret = (prime + secret + (yj * lagrange_coefficient)) % prime
    return secret

# مثال على الاستخدام
if __name__ == "__main__":
    secret = 1234  # السر المراد مشاركته
    k = 3          # الحد الأدنى لإعادة بناء السر
    n = 5          # عدد الحصص الإجمالي
    prime = 1613   # عدد أولي أكبر من السر

    shares = generate_shares(secret, k, n, prime)
    print("الحصص المولدة:", shares)

    # إعادة بناء السر باستخدام أي k حصص
    selected_shares = random.sample(shares, k)
    reconstructed_secret = reconstruct_secret(selected_shares, prime)
    print("السر المعاد بناءه:", reconstructed_secret)


الحصص المولدة: [(1, 67), (2, 112), (3, 1369), (4, 612), (5, 1067)]
السر المعاد بناءه: 1234
